#TEXT CLUSTERING


In [1]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [2]:
import requests
import json
'''
headers = {
    "Authorization": f"Bearer vmMtGjKoFFc65vjv2cKDNj70vHbOeUJcN6GgWPJ5eeA"
}
'''


response = requests.get(f"https://mastodon.social/api/v1/timelines/public?limit=40")

# Get the JSON data from the response
data = response.json()



In [33]:
#data_frame =json.loads(response.text)
#data_frame

CLEANING THE DATA

In [12]:
import json
import csv
import re
from html import unescape

def clean_content(content):
    # Remove HTML tags
    clean = re.sub(r'<[^>]+>', '', content)
    # Unescape HTML entities
    clean = unescape(clean)
    # Remove URLs
    clean = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', clean)
    # Remove emojis and special characters
    clean = re.sub(r'[^\w\s.,!?]', '', clean)
    # Remove extra whitespace
    clean = ' '.join(clean.split())
    # Remove timestamps, dates, and other non-essential information
    clean = re.sub(r'\d{1,2}:\d{2}(-\d{1,2}:\d{2})?(\s+UTC)?', '', clean)
    clean = re.sub(r'\b(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday)\b', '', clean)
    clean = re.sub(r'\d{1,2}(st|nd|rd|th)', '', clean)
    # Remove any remaining parentheses and their contents
    clean = re.sub(r'\([^)]*\)', '', clean)
    # Final cleanup of extra spaces
    clean = ' '.join(clean.split())
    return clean.strip()

# Load the JSON data
data = json.loads(response.text)

# Prepare data for CSV
csv_data = []
for item in data:
    if item.get('language') == 'en':
        cleaned_content = clean_content(item['content'])
        # Only include non-empty content
        if cleaned_content:
            csv_data.append({
                'id': item['id'],
                'language': item['language'],
                'content': cleaned_content
            })

# Write to CSV
with open('cleaned_output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['id', 'language', 'content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in csv_data:
        writer.writerow(row)

print("CSV file 'cleaned_output.csv' has been created successfully.")

CSV file 'cleaned_output.csv' has been created successfully.


In [13]:
df=pd.read_csv('cleaned_output.csv')
df

,id,language,content
0,113171522990868526,en,Google employees attempts to hide messages fro...
1,113171522923192658,en,New APK Google Drive 2.24.367.5.all by Google LLC
2,113171522889258215,en,"for all the motorcycle folk on here, how would..."
3,113171522879043094,en,New APK Google Drive 2.24.377.4.all by Google LLC
4,113171522871114137,en,Virat Kohlis Test average falls to eightyear l...
5,113171522849668217,en,Φωτιά ΧΥΤΑ_Καρπάθου Φωτιά στο ΧΥΤΑ Καρπάθου
6,113171522844025296,en,Hezbollahs Big Rocket Attack Minutes After IDF...
7,113171522807752059,en,"I eveny him, wish I could be him Ashwin hails ..."
8,113171522764678508,en,Imran Khans party gets permit for rally after ...
9,113171522734576065,en,Halle Berry surprises fans at special wig scre...


In [9]:
pip install transformers scikit-learn torch


In [10]:
!pip install spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 107.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


1ST APPORACH USING TEXT CLUSTERING ALGORITHM(Hierarchical clustering)

In [32]:
import spacy
from transformers import BertTokenizer, BertModel
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import silhouette_score
import torch
import numpy as np

# Step 1: Use BERT to convert each sentence into a vector
class BERTEmbedding:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')

    def encode(self, sentence):
        # Tokenize and encode sentence
        inputs = self.tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)

        # Get the sentence embedding (average of the last hidden states)
        sentence_embedding = torch.mean(outputs.last_hidden_state, dim=1)
        return sentence_embedding.squeeze().numpy()

# Step 2: Use SpaCy to recognize Named Entities (NER) and filter meaningful keywords
nlp = spacy.load("en_core_web_sm")

def extract_named_entities_and_keywords(sentence):
    doc = nlp(sentence)

    # Extract named entities (like persons, organizations)
    entities = [ent.text for ent in doc.ents if ent.label_ in ('PERSON', 'ORG', 'GPE')]

    # Combine multi-word entities into single entities
    combined_entities = combine_multi_word_entities(entities)

    # Extract non-adjective keywords (ignoring "good", "very", etc.), filtering out entities
    keywords = [token.text for token in doc if token.pos_ in ('NOUN', 'PROPN') and token.text not in combined_entities]

    return combined_entities, keywords

# Function to combine multi-word entities (e.g., "Banotu Ganesh")
def combine_multi_word_entities(entities):
    multi_word_entities = []
    for entity in entities:
        if ' ' in entity:
            multi_word_entities.append(entity)
        else:
            if not any(entity in e for e in multi_word_entities):
                multi_word_entities.append(entity)
    return list(set(multi_word_entities))

# Step 3: Apply Hierarchical Clustering to group similar sentences into clusters
def cluster_sentences(sentences, n_clusters=None):
    bert_embedding = BERTEmbedding()
    embeddings = np.array([bert_embedding.encode(sentence) for sentence in sentences])

    # Compute pairwise cosine distances
    distances = cosine_distances(embeddings)

    # Apply Agglomerative Clustering
    clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage='complete')
    labels = clustering.fit_predict(distances)

    return clustering, labels

# Step 4: Extract key terms for each cluster
def extract_cluster_keywords(sentences, labels):
    cluster_keywords = {}
    unique_labels = set(labels)

    for label in unique_labels:
        cluster_sentences = [sentences[i] for i in range(len(sentences)) if labels[i] == label]
        cluster_entities_keywords = []

        for sentence in cluster_sentences:
            entities, keywords = extract_named_entities_and_keywords(sentence)
            cluster_entities_keywords.extend(entities + keywords)

        # Deduplicate and store the entities/keywords for the cluster
        cluster_keywords[label] = list(set(cluster_entities_keywords))

    return cluster_keywords

# Example usage
if __name__ == "__main__":
    # Sample list of sentences to cluster (replace this with your actual data)
    sentences = [
        "Halle Berry surprises fans at special wig screening of her upcoming movie.",
        "The movie Never Let Go is released today.",
        "Berry donned her iconic hairstyle from her 2013 film, The Call.",
        "New technology is changing the way we interact with films.",
        # Add more sentences as needed...
    ]

    # Step 1: Determine the best number of clusters using silhouette score
    best_n_clusters = 0
    best_score = -1
    bert_embedding = BERTEmbedding()  # Initialize BERTEmbedding once for use

    for n in range(2, len(sentences)):  # Testing different cluster sizes
        # only test for number of clusters less than the number of sentences
        clustering, labels = cluster_sentences(sentences, n)
        score = silhouette_score(cosine_distances(np.array([bert_embedding.encode(s) for s in sentences])), labels)

        if score > best_score:
            best_score = score
            best_n_clusters = n

    print(f"Best number of clusters: {best_n_clusters} with silhouette score: {best_score}")

    # Step 2: Cluster sentences using the best number of clusters
    clustering, labels = cluster_sentences(sentences, best_n_clusters)

    # Step 3: Extract key terms for each cluster
    cluster_keywords = extract_cluster_keywords(sentences, labels)
    print(f"Cluster Keywords: {cluster_keywords}")

    # Step 4: Predict the category for a new sentence based on cluster membership
    new_sentence = "Halle Berry surprises fans at special wig screening of her upcoming movie Never Let Go."
    entities, keywords = extract_named_entities_and_keywords(new_sentence)

    # Combine entities and keywords, remove duplicates
    categories = list(set(entities + keywords))

    print(f"Sentence: {new_sentence}")
    print(f"Groups belong to: {', '.join(categories)}")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-package

Best number of clusters: 2 with silhouette score: 0.1789727807044983


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Cluster Keywords: {0: ['technology', 'today', 'way', 'movie', 'films'], 1: ['Halle Berry', 'film', 'wig', 'fans', 'Halle', 'Call', 'Berry', 'movie', 'hairstyle', 'screening']}
Sentence: Halle Berry surprises fans at special wig screening of her upcoming movie Never Let Go.
Groups belong to: Halle Berry, wig, fans, Halle, Berry, movie, screening


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_agglomerative.py:1006: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


SECOND APPORACH USING BertModel AND spacy

In [14]:
#SECOND CODE
import spacy
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Step 1: Use BERT to convert each sentence into a vector
class BERTEmbedding:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertModel.from_pretrained('bert-base-uncased')

    def encode(self, sentence):
        # Tokenize and encode sentence
        inputs = self.tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = self.model(**inputs)

        # Get the sentence embedding (average of the last hidden states)
        sentence_embedding = torch.mean(outputs.last_hidden_state, dim=1)
        return sentence_embedding.squeeze().numpy()

# Step 2: Use SpaCy to recognize Named Entities (NER) and filter meaningful keywords
nlp = spacy.load("en_core_web_sm")

def extract_named_entities_and_keywords(sentence):
    doc = nlp(sentence)

    # Extract named entities (like persons, organizations, places)
    entities = [ent.text for ent in doc.ents if ent.label_ in ('PERSON', 'ORG', 'GPE')]

    # Combine multi-word entities into single entities
    combined_entities = combine_multi_word_entities(entities)

    # Extract meaningful keywords (nouns) and filter out entities
    keywords = [token.text for token in doc if token.pos_ == 'NOUN' and token.text not in combined_entities]

    return combined_entities, keywords

# Function to combine multi-word entities (e.g., "Banotu Ganesh", "Rahul Dindigal")
def combine_multi_word_entities(entities):
    multi_word_entities = []
    for entity in entities:
        # Keep the entity intact if it's multi-word
        if ' ' in entity:
            multi_word_entities.append(entity)
        else:
            # Avoid partial matches for single-word entities
            if not any(entity in e for e in multi_word_entities):
                multi_word_entities.append(entity)
    return list(set(multi_word_entities))

# Example usage
if __name__ == "__main__":
    for i in df['content']:
      new_sentence_2 = i
    # Use NER to extract proper entities and keywords from the sentence
      entities_2, keywords_2 = extract_named_entities_and_keywords(new_sentence_2)

    # Combine entities and keywords, ensuring no duplicates
      categories_2 = list(set(entities_2 + keywords_2))

    # Output the final groups, formatted as expected
      print(f"\nSentence: {new_sentence_2}")
      print(f"Groups belong to: {', '.join(categories_2)}\n")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)



Sentence: Google employees attempts to hide messages from investigators might backfire Posted into Features from The Verge featuresfromthevergetheverge
Groups belong to: employees, investigators, featuresfromthevergetheverge, Google, Features, messages


Sentence: New APK Google Drive 2.24.367.5.all by Google LLC
Groups belong to: Google


Sentence: for all the motorcycle folk on here, how would you have taken this strange little railway bridge on a backroad?motorcyclesmotorrad
Groups belong to: railway, folk, backroad?motorcyclesmotorrad, motorcycle, bridge


Sentence: New APK Google Drive 2.24.377.4.all by Google LLC
Groups belong to: Google


Sentence: Virat Kohlis Test average falls to eightyear low In both innings of the Test, Kohli delivered poor scores of 6 and 17. While in the first innings, he chased an outside off stump delivery by Hasan Mahmud, in the second innings, he fell victim to a wrong legbeforewicket decision. Having chosen to not review it, the Ultraedge discovered

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
